In [13]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array

from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle

import pyswarms as ps
import numpy as np
import pandas as pd
import os

In [2]:
def create_street_data(path, street_types, im_size):
    images, labels = [], []
    streets = [(item, os.path.join(path, item, street)) 
               for item in street_types 
               for street in os.listdir(os.path.join(path, item))]
    streets_df = pd.DataFrame(streets, columns=['street type', 'image'])
    
    for _, row in streets_df.iterrows():
        img = load_img(row['image'], target_size=(im_size, im_size))
        images.append(img_to_array(img))
        labels.append(row['street type'])
    
    return np.array(images, dtype='float32') / 255.0, np.array(labels)

In [3]:
# Define o tamanho da imagem e os tipos de rua
im_size = 350  # Exemplo de tamanho de imagem

"""
street_types = ['Apple', 'Banana', 'Cocos']
path = '../DatasetFruits/'
path_test = '../DatasetFruits_Test/'
"""

street_types = ['clean', 'litter', 'recycle']
path = '../Dataset/'
path_test = '../Dataset_Test/'

In [4]:
# Criando dados de treino, validação e teste
train_images, train_labels = create_street_data(path, street_types, im_size)
test_images, test_labels = create_street_data(path_test, street_types, im_size)

# Contando as ocorrências de cada tipo de rua e imprimindo
streets_count = pd.value_counts(train_labels)
print("Streets in each category:", streets_count)

Streets in each category: recycle    1675
clean      1625
litter     1505
dtype: int64


In [5]:
# Convertendo etiquetas para valores numéricos
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Embaralhando e dividindo os dados de treino e validação
train_x, val_x, train_y, val_y = train_test_split(train_images, train_labels_encoded, test_size=0.2, random_state=415)

# Imprimindo as formas dos conjuntos de dados
print(f"Train shape: {train_x.shape}")
print(f"Validation shape: {val_x.shape}")
print(f"Test shape: {test_images.shape}")

Train shape: (3844, 350, 350, 3)
Validation shape: (961, 350, 350, 3)
Test shape: (600, 350, 350, 3)


In [20]:
# Defina as funções de treinamento de rede e fitness para o PSO
def train_network(hyperparameters, train_x, train_y, val_x, val_y):
    learning_rate, dropout_rate = hyperparameters
    
    # Definindo o modelo da rede neural convolucional
    model = Sequential([
        Conv2D(64, (3, 3), activation='relu', input_shape=(im_size, im_size, 3)),
        MaxPooling2D((2, 2)),
        
        Flatten(),
        
        Dropout(dropout_rate),
        
        Dense(len(street_types), activation='softmax')   
    ])
    
    model.compile(optimizer=Adam(slearning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    model.fit(train_x, 
              train_y, 
              epochs=5,
              validation_data=(val_x, val_y), 
              verbose=0)
    
    validation_loss, validation_accuracy = model.evaluate(val_x, val_y, verbose=0)
    
    return validation_loss

In [19]:
def fitness_function(x, train_x, train_y, val_x, val_y):
    
    n_particles = x.shape[0]
    
    losses = []
    
    for i in range(n_particles):
        # x[i] contém os hiperparâmetros para a i-ésima partícula
        hyperparameters = x[i]
        loss = train_network(hyperparameters, train_x, train_y, val_x, val_y)
        losses.append(loss)
        
    return np.array(losses)

In [8]:
# Inicializando o otimizador GlobalBestPSO do pyswarms
bounds = [(0.0001, 0.1), (0.0, 0.5)]  # (min_learning_rate, max_learning_rate), (min_dropout_rate, max_dropout_rate)
options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}

optimizer = ps.single.GlobalBestPSO(n_particles=5, 
                                    dimensions=2, 
                                    options=options, 
                                    bounds=bounds)

# Executando o PSO para encontrar os melhores hiperparâmetros
cost, best_pos = optimizer.optimize(fitness_function, 
                                    iters=5, 
                                    train_x=train_x, 
                                    train_y=train_y, 
                                    val_x=val_x, 
                                    val_y=val_y)

best_learning_rate, best_dropout_rate = best_pos

2023-11-08 11:44:11,151 - pyswarms.single.global_best - INFO - Optimize for 5 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|5/5, best_cost=0.196
2023-11-08 16:33:36,606 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.19598796963691711, best pos: [0.00018259 0.10997558]


In [10]:
final_model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(im_size, im_size, 3)),
    MaxPooling2D((2, 2)),

    Flatten(),

    Dropout(best_dropout_rate),

    Dense(len(street_types), activation='softmax')    
])

final_model.compile(optimizer=Adam(best_learning_rate),
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])

history = final_model.fit(train_x, 
                          train_y, 
                          epochs=5,
                          validation_data=(val_x, val_y))

Epoch 1/5
121/121 [==============================] - 130s 1s/step - loss: 1.9571 - accuracy: 0.6178 - val_loss: 0.4793 - val_accuracy: 0.8283
Epoch 2/5
121/121 [==============================] - 136s 1s/step - loss: 0.2733 - accuracy: 0.9030 - val_loss: 0.3185 - val_accuracy: 0.8835
Epoch 3/5
121/121 [==============================] - 139s 1s/step - loss: 0.1069 - accuracy: 0.9740 - val_loss: 0.2301 - val_accuracy: 0.9334
Epoch 4/5
121/121 [==============================] - 139s 1s/step - loss: 0.0509 - accuracy: 0.9919 - val_loss: 0.2401 - val_accuracy: 0.9303
Epoch 5/5
121/121 [==============================] - 143s 1s/step - loss: 0.0303 - accuracy: 0.9971 - val_loss: 0.2104 - val_accuracy: 0.9407


In [15]:
test_loss, test_accuracy = final_model.evaluate(test_images, test_labels_encoded, verbose=1);

y_pred_test = final_model.predict(test_images);
y_pred_test_classes = np.argmax(y_pred_test, axis=1);

y_true = test_labels_encoded;
y_true_binarized = label_binarize(test_labels_encoded, classes=np.unique(test_labels_encoded));

19/19 [==============================] - 3s 162ms/step
Best learning rate: 0.00018258859132538424
Best dropout rate: 0.10997558109383135


In [17]:
print(f"Best learning rate: {best_learning_rate}")
print(f"Best dropout rate: {best_dropout_rate}")

# Avaliação do modelo nos dados de teste
print(f"\nTest accuracy: {test_accuracy*100:.2f}%" + "\n");

# Matriz de confusão
confusion_mtx = confusion_matrix(y_true, y_pred_test_classes)
print("Confusion Matrix:")
print(confusion_mtx)

# Relatório de classificação
class_report = classification_report(y_true, y_pred_test_classes, target_names=label_encoder.classes_)
print("\nClassification Report:")
print(class_report)
print("\n")

# Calculando o AUC para cada classe
for i in range(y_true_binarized.shape[1]):  # Percorre cada classe
    auc_score = roc_auc_score(y_true_binarized[:, i], y_pred_test[:, i])
    print(f'AUC for class {label_encoder.classes_[i]}: {auc_score:.2f}')

Best learning rate: 0.00018258859132538424
Best dropout rate: 0.10997558109383135

Test accuracy: 92.50%

Confusion Matrix:
[[176  20   4]
 [  8 192   0]
 [ 12   1 187]]

Classification Report:
              precision    recall  f1-score   support

       clean       0.90      0.88      0.89       200
      litter       0.90      0.96      0.93       200
     recycle       0.98      0.94      0.96       200

    accuracy                           0.93       600
   macro avg       0.93      0.92      0.93       600
weighted avg       0.93      0.93      0.93       600



AUC for class clean: 0.98
AUC for class litter: 0.99
AUC for class recycle: 0.99
